In [1]:
import findspark
import pyspark
import os
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('local').getOrCreate()

from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import to_date, to_timestamp

In [2]:
#数据路径
path1 = r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\csvFile\\ad_static_feature.csv'
path2 = r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\csvFile\\ad_operation.csv'
path3 = r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\user\\user_data'
path4 = r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\imps_log\\totalExposureLog.out'
path5 = r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\csvFile\\test_sample.csv'

### 第一步、处理df_ad_static

In [3]:
####------------df_ad_static---------
df_ad_static_feature=spark.read.csv(path1,header=True)
df_ad_static_feature.select('create_time').show(3)
df_ad_static_feature.show(30)
df_ad_static_feature = df_ad_static_feature.withColumn('create_date', from_unixtime('create_time','yyyy-MM-dd HH:mm:ss'))
df_ad_static_feature.show()

+-----------+
|create_time|
+-----------+
| 1550991780|
| 1552296016|
| 1550823156|
+-----------+
only showing top 3 rows

+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
| ad_id|create_time|ad_account_id|goods_id|goods_types|industry_id|ad_size|        create_date|
+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
|144732| 1550991780|        13937|      -1|          1|        186|     14|2019-02-24 15:03:00|
| 70793| 1552296016|         6932|      -1|          1|         21|     14|2019-03-11 17:20:16|
|467150| 1550823156|          171|   21922|         18|        232|     30|2019-02-22 16:12:36|
|383839| 1551188776|        10704|    4242|          8|        147|      3|2019-02-26 21:46:16|
|245464| 1550459042|        26317|   32825|          5|         47|     44|2019-02-18 11:04:02|
|629739| 1552123718|        14159|   32101|         18|         13|     36|2019-03-09 17:28:38|
|505234| 1515

In [4]:
##去重复
df_ad_static_feature = df_ad_static_feature.dropDuplicates()


In [5]:
df_ad_static_feature.show(5)

+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
| ad_id|create_time|ad_account_id|goods_id|goods_types|industry_id|ad_size|        create_date|
+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
|144732| 1550991780|        13937|      -1|          1|        186|     14|2019-02-24 15:03:00|
| 70793| 1552296016|         6932|      -1|          1|         21|     14|2019-03-11 17:20:16|
|467150| 1550823156|          171|   21922|         18|        232|     30|2019-02-22 16:12:36|
|383839| 1551188776|        10704|    4242|          8|        147|      3|2019-02-26 21:46:16|
|245464| 1550459042|        26317|   32825|          5|         47|     44|2019-02-18 11:04:02|
+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
only showing top 5 rows



In [6]:
##去缺失值
import pyspark.sql.functions as fn
df_ad_static_feature.agg(*[(1-(fn.count(c)/fn.count("*"))).alias(c+'_missing')
                           for c in df_ad_static_feature.columns]).show()
df_ad_static_feature = df_ad_static_feature.dropna()

+-------------+-------------------+---------------------+----------------+-------------------+-------------------+---------------+-------------------+
|ad_id_missing|create_time_missing|ad_account_id_missing|goods_id_missing|goods_types_missing|industry_id_missing|ad_size_missing|create_date_missing|
+-------------+-------------------+---------------------+----------------+-------------------+-------------------+---------------+-------------------+
|          0.0|                0.0|                  0.0|             0.0|                0.0|                0.0|            0.0|                0.0|
+-------------+-------------------+---------------------+----------------+-------------------+-------------------+---------------+-------------------+



In [7]:
##去异常值
"""industry_id出现多值的情况"""
df_ad_static_feature = df_ad_static_feature.toPandas()
def is_outer(x):
    if ',' in x:
        return '1'
    else:
        return '0'
df_ad_static_feature['is_industry_outer'] = df_ad_static_feature['industry_id'].map(lambda x : is_outer(x))
df_ad_static_feature = df_ad_static_feature[df_ad_static_feature['is_industry_outer']=='0']
del df_ad_static_feature['is_industry_outer']
df_ad_static_feature.head()

,ad_id,create_time,ad_account_id,goods_id,goods_types,industry_id,ad_size,create_date
0,144732,1550991780,13937,-1,1,186,14,2019-02-24 15:03:00
1,70793,1552296016,6932,-1,1,21,14,2019-03-11 17:20:16
2,467150,1550823156,171,21922,18,232,30,2019-02-22 16:12:36
3,383839,1551188776,10704,4242,8,147,3,2019-02-26 21:46:16
4,245464,1550459042,26317,32825,5,47,44,2019-02-18 11:04:02


In [8]:
#将处理好的数据保存
#df_ad_static_feature.to_csv( r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\csvFile\\ad_static_feature.csv',header = True,index=False)

In [9]:
#读取处理好的数据
df_ad_static_feature=spark.read.csv(r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\csvFile\\ad_static_feature.csv',header=True)

In [10]:
df_ad_static_feature.show(5)

+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
| ad_id|create_time|ad_account_id|goods_id|goods_types|industry_id|ad_size|        create_date|
+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
|144732| 1550991780|        13937|      -1|          1|        186|     14|2019-02-24 15:03:00|
| 70793| 1552296016|         6932|      -1|          1|         21|     14|2019-03-11 17:20:16|
|467150| 1550823156|          171|   21922|         18|        232|     30|2019-02-22 16:12:36|
|383839| 1551188776|        10704|    4242|          8|        147|      3|2019-02-26 21:46:16|
|245464| 1550459042|        26317|   32825|          5|         47|     44|2019-02-18 11:04:02|
+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
only showing top 5 rows



In [11]:
df_ad_static_feature.count()

502156

In [12]:
##一点描述性统计
#goods_types
df_ad_static_feature.groupby('goods_types').count().sort('count',ascending=False).show()
"""推广商品类型为18的广告最多"""

+-----------+------+
|goods_types| count|
+-----------+------+
|         18|182330|
|          1|100833|
|          8| 60693|
|          5| 56746|
|          3| 52688|
|         13| 38621|
|          2|  9208|
|         15|   833|
|         12|    97|
|         14|    96|
|          7|    11|
+-----------+------+



'推广商品类型为18的广告最多'

In [13]:
df_ad_static_feature.describe('ad_size').show()

+-------+------------------+
|summary|           ad_size|
+-------+------------------+
|  count|            502156|
|   mean| 43.33296492118967|
| stddev|18.320716202699142|
|    min|                 1|
|    max|                 9|
+-------+------------------+



In [14]:
df_ad_static_feature.groupby('ad_size').count().sort('count',ascending=False).show()

+-------+------+
|ad_size| count|
+-------+------+
|     64|158523|
|     36| 76462|
|     40| 51689|
|     44| 45556|
|     30| 43772|
|      1| 27337|
|     14| 25096|
|     57| 19030|
|     34| 12367|
|     54| 11759|
|     50|  6985|
|     20|  4335|
|     19|  3844|
|     43|  2869|
|     25|  2025|
|     37|  1646|
|      3|  1048|
|     29|  1006|
|     26|   933|
|     13|   796|
+-------+------+
only showing top 20 rows



### 第二步、处理df_ad_operation

In [15]:
####------------df_ad_operation---------
df_ad_operation=spark.read.csv(path2,header=True)
df_ad_operation=df_ad_operation.withColumnRenamed('aft_chg',' aft_chg_')
#异常时间
df_ad_operation.filter(df_ad_operation.crt_chg_time  =='20190230000000').show()

+------+--------------+-------+-------+---------+
| ad_id|  crt_chg_time|crt_chg|chg_typ| aft_chg_|
+------+--------------+-------+-------+---------+
|593323|20190230000000|      1|      1|        0|
|146563|20190230000000|      1|      1|        0|
|493936|20190230000000|      1|      1|        0|
|180239|20190230000000|      1|      1|        0|
| 80499|20190230000000|      1|      1|        0|
|135889|20190230000000|      1|      1|        0|
|587541|20190230000000|      1|      1|        0|
|137628|20190230000000|      1|      1|        0|
|199142|20190230000000|      1|      1|        0|
|606740|20190230000000|      1|      1|        0|
|650468|20190230000000|      1|      1|        0|
|712452|20190230000000|      1|      1|        0|
|600160|20190230000000|      1|      1|        0|
|191049|20190230000000|      1|      1|        0|
|439612|20190230000000|      1|      1|        0|
|370817|20190230000000|      1|      1|        0|
|515677|20190230000000|      1|      1|        0|


In [16]:
#去除异常时间
df_ad_operation = df_ad_operation.filter(df_ad_operation.crt_chg_time  !='20190230000000')

In [17]:
#转时间
date = df_ad_operation.select(to_timestamp(df_ad_operation.crt_chg_time,'yyyyMMddHHmmss').alias('crt_chg_date'))
df_ad_operation = df_ad_operation.withColumn("crt_chg_date",to_timestamp(df_ad_operation.crt_chg_time,'yyyyMMddHHmmss'))

In [18]:
#把表拉长
from pyspark.sql import functions as F       
df_ad_operation = df_ad_operation.withColumn("ad_bid",F.when(df_ad_operation.chg_typ == '2',df_ad_operation[' aft_chg_']))
df_ad_operation = df_ad_operation.withColumn("target_user",F.when(df_ad_operation.chg_typ == '3',df_ad_operation[' aft_chg_']))
df_ad_operation = df_ad_operation.withColumn("put_time",F.when(df_ad_operation.chg_typ == '4',df_ad_operation[' aft_chg_']))
df_ad_operation = df_ad_operation.withColumn("ad_state",F.when(df_ad_operation.chg_typ == '1',df_ad_operation[' aft_chg_']))
df_ad_operation.show(15)

+------+--------------+-------+-------+--------------------+-------------------+------+--------------------+--------------------+--------+
| ad_id|  crt_chg_time|crt_chg|chg_typ|            aft_chg_|       crt_chg_date|ad_bid|         target_user|            put_time|ad_state|
+------+--------------+-------+-------+--------------------+-------------------+------+--------------------+--------------------+--------+
|593323|             0|      2|      2|                  90|               null|    90|                null|                null|    null|
|593323|             0|      2|      3|                 all|               null|  null|                 all|                null|    null|
|593323|             0|      2|      4|281474976710655,2...|               null|  null|                null|281474976710655,2...|    null|
|593323|20190217000000|      1|      1|                   0|2019-02-17 00:00:00|  null|                null|                null|       0|
|593323|20190218233855|    

In [19]:
##---------转化为pandas来做---------
#ad_operation = df_ad_operation.toPandas()
#ad_operation['crt_chg_date'] = ad_operation['crt_chg_date'].fillna('0')
##去异常状态修改超过100次的认为是异常的
#outer_id = set(chg_state_count[chg_state_count['chg_state_count']>100]['ad_id'])
#def detec_outer(x,outer_id):
#    if x in outer_id:
#        return True
#    else:
#        return False
#ad_operation['isouter'] = ad_operation['ad_id'].map(lambda x : detec_outer(x,outer_id))
#ad_operation = ad_operation[ad_operation['isouter']==False]
#ad_operation.drop('isouter',axis=1,inplace=True)
##填充缺失值，向前,向后填充
#ffill_df = ad_operation.groupby(['ad_id']).fillna(method='ffill')
#columns = ['ad_id']
#ffill_tmp = pd.concat((ad_operation[columns ],ffill_df),axis=1)
#bfill_df = ffill_tmp.groupby(['ad_id']).fillna(method='bfill')
#bfill_tmp = pd.concat((ad_operation[columns ],bfill_df),axis=1)
#去重复
#bfill_tmp.drop_duplicates(inplace=True)
#保存数据
#bfill_tmp.to_csv(r'D:\AD_ALG\ad_operation_clear.csv',encoding='utf-8-sig',index=False,header=True)

In [20]:
#经过pandas处理后最终的数据为
df_ad_operation=spark.read.csv(r'D:\AD_ALG\ad_operation_clear.csv',header=True)
df_ad_operation.show(10)

+-----+------+--------+-------+-------+-------------------+--------------+--------------------+-----------+
|ad_id|ad_bid|ad_state|chg_typ|crt_chg|       crt_chg_date|  crt_chg_time|            put_time|target_user|
+-----+------+--------+-------+-------+-------------------+--------------+--------------------+-----------+
|   31|   100|       1|    234|      2|2019-03-12 23:22:50|             0|281474976694272,2...| area:11442|
|   31|   100|       0|      1|      1|2019-03-13 19:52:07|20190313195207|281474976694272,2...| area:11442|
|   32|    83|       1|    234|      2|2018-11-30 07:40:17|             0|70368475742208,70...|  area:7572|
|   32|    91|       1|      2|      1|2019-02-21 05:31:31|20190221053131|70368475742208,70...|  area:7572|
|   32|    91|       0|      1|      1|2019-02-21 22:34:03|20190221223403|70368475742208,70...|  area:7572|
|   32|    91|       1|      1|      1|2019-02-22 05:31:39|20190222053139|70368475742208,70...|  area:7572|
|   32|    91|       0|     

### 第三步、处理df_user_data

In [21]:
#df_user_data
df_user_data=spark.read.csv(path3,sep='\t',header=None)
df_user_data = df_user_data.withColumnRenamed('_c0','user_id').withColumnRenamed('_c1','age').withColumnRenamed('_c2','gender').withColumnRenamed('_c3','area*').withColumnRenamed('_c4','status*').withColumnRenamed('_c5','education').withColumnRenamed('_c6','consuption_ability').withColumnRenamed('_c7','device').withColumnRenamed('_c8','work*').withColumnRenamed('_c9','connectiontype').withColumnRenamed('_c10','behavior*')
df_user_data.show(5)

+-------+---+------+--------------------+-------+---------+------------------+------+-----+--------------+--------------------+
|user_id|age|gender|               area*|status*|education|consuption_ability|device|work*|connectiontype|           behavior*|
+-------+---+------+--------------------+-------+---------+------------------+------+-----+--------------+--------------------+
| 624218|753|     2|523,7589,12527,90...|      0|        6|                 1|     2|    0|             4|32612,34406,3413,...|
| 271475|601|     3|9019,9032,663,125...|     13|        7|                 2|     2|    0|             2|4990,4992,3413,33...|
|  44439|819|     3|12527,14188,9841,...|     13|        7|                 3|     2|    4|             4|3099,4403,34414,4...|
| 863717|  1|     2|12527,3435,2398,1...|      6|        5|                 3|     2|    0|             2|34293,845,3413,30...|
|1305469|714|     3|8928,9019,1540,71...|     13|        5|                 3|     2|    0|             

In [22]:
df_user_data.describe()

DataFrame[summary: string, user_id: string, age: string, gender: string, area*: string, status*: string, education: string, consuption_ability: string, device: string, work*: string, connectiontype: string, behavior*: string]

In [23]:
#一些描述性统计
df_user_data.describe().select('summary','age','gender','connectiontype','device','education','consuption_ability').show()

+-------+-----------------+------------------+------------------+-------------------+-----------------+------------------+
|summary|              age|            gender|    connectiontype|             device|        education|consuption_ability|
+-------+-----------------+------------------+------------------+-------------------+-----------------+------------------+
|  count|          1396718|           1396718|           1396718|            1396718|          1396718|           1396718|
|   mean|526.3470013273975|2.4997451167665914|2.8176883236272463|  2.186646123268978| 4.77900406524438| 2.161956099942866|
| stddev|280.6423081629517|   0.5654588072063| 1.025974199358969|0.39513780269948995|2.086811161155874|0.8310177086340266|
|    min|                0|                 1|                 1|                  0|                1|                 1|
|    max|              988|                 3|                 5|                  4|                8|                 3|
+-------+-------

In [24]:
#gender
df_user_data.groupby('gender').count().sort('count',ascending=False).show()
df_user_data.groupby('connectiontype').count().sort('count',ascending=False).show()
df_user_data.groupby('device').count().sort('count',ascending=False).show()
df_user_data.groupby('education').count().sort('count',ascending=False).show()
df_user_data.groupby('consuption_ability').count().sort('count',ascending=False).show()

+------+------+
|gender| count|
+------+------+
|     3|746709|
|     2|601303|
|     1| 48706|
+------+------+

+--------------+------+
|connectiontype| count|
+--------------+------+
|             2|819197|
|             4|503779|
|             5| 39403|
|             3| 25326|
|             1|  9013|
+--------------+------+

+------+-------+
|device|  count|
+------+-------+
|     2|1133994|
|     3| 261388|
|     0|    842|
|     4|    494|
+------+-------+

+---------+------+
|education| count|
+---------+------+
|        5|349100|
|        2|337743|
|        6|258055|
|        8|157753|
|        7|150252|
|        3|114473|
|        1| 22990|
|        4|  6352|
+---------+------+

+------------------+------+
|consuption_ability| count|
+------------------+------+
|                 3|613701|
|                 2|395523|
|                 1|387494|
+------------------+------+



### 第四步、处理f_totalExposureLog

In [25]:
#df_totalExposureLog
df_totalExposureLog = spark.read.csv(path4,sep='\t',header=None) 
df_totalExposureLog = df_totalExposureLog.withColumnRenamed('_c0','ad_rqst_id').withColumnRenamed('_c1','ad_rqst_time').withColumnRenamed('_c2','ad_place_id').withColumnRenamed('_c3','user_id').withColumnRenamed('_c4','ad_id').withColumnRenamed('_c5','ad_size').withColumnRenamed('_c6','ad_bid').withColumnRenamed('_c7',' pctr').withColumnRenamed('_c8','quality_ecpm').withColumnRenamed('_c9','totalEcpm')
df_totalExposureLog = df_totalExposureLog.withColumn('ad_rqst_date', from_unixtime('ad_rqst_time','yyyy-MM-dd HH:mm:ss'))
df_totalExposureLog.show(5)

+----------+------------+-----------+-------+------+-------+------+------+------------+---------+-------------------+
|ad_rqst_id|ad_rqst_time|ad_place_id|user_id| ad_id|ad_size|ad_bid|  pctr|quality_ecpm|totalEcpm|       ad_rqst_date|
+----------+------------+-----------+-------+------+-------+------+------+------------+---------+-------------------+
|  53991770|  1550409746|         94|1160618|451525|     50|    46|47.217|      944.34|  3122.34|2019-02-17 21:22:26|
|  25942318|  1550370892|         79| 203814|214797|     64|    10|49.094|      981.88|  1471.88|2019-02-17 10:34:52|
|  66156247|  1550416600|         18| 808543| 92253|     40|    96| 3.824|       76.48|  443.584|2019-02-17 23:16:40|
|   5935886|  1550365898|        198|   7270|160082|     64|    85| 6.123|      122.46|  642.915|2019-02-17 09:11:38|
|  11624425|  1550361159|        168| 852707|253902|     64|    60| 4.329|       86.58|   346.32|2019-02-17 07:52:39|
+----------+------------+-----------+-------+------+----

In [26]:
#去缺失值
df_totalExposureLog = df_totalExposureLog.dropna()

In [27]:
df_totalExposureLog =df_totalExposureLog.dropDuplicates(subset=['ad_rqst_id','user_id','ad_id'])

In [28]:
df_totalExposureLog.show(3)

+----------+------------+-----------+-------+------+-------+------+------+------------+---------+-------------------+
|ad_rqst_id|ad_rqst_time|ad_place_id|user_id| ad_id|ad_size|ad_bid|  pctr|quality_ecpm|totalEcpm|       ad_rqst_date|
+----------+------------+-----------+-------+------+-------+------+------+------------+---------+-------------------+
|  10000035|  1552305125|         18| 567406|624183|     40|    49|19.843|      396.86| 1369.167|2019-03-11 19:52:05|
|  10000098|  1551536152|         39| 816739|534452|     36|    45|12.157|      243.14|   790.14|2019-03-02 22:15:52|
|  10000119|  1551093547|         25| 558284|157969|     30|    28|21.044|      420.88| 1010.112|2019-02-25 19:19:07|
+----------+------------+-----------+-------+------+-------+------+------+------------+---------+-------------------+
only showing top 3 rows



In [29]:
from pyspark.sql.functions import to_date
df_totalExposureLog  = df_totalExposureLog.withColumn('ad_rqst_date',to_date(df_totalExposureLog.ad_rqst_date))

In [30]:
df_totalExposureLog.show(3)

+----------+------------+-----------+-------+------+-------+------+------+------------+---------+------------+
|ad_rqst_id|ad_rqst_time|ad_place_id|user_id| ad_id|ad_size|ad_bid|  pctr|quality_ecpm|totalEcpm|ad_rqst_date|
+----------+------------+-----------+-------+------+-------+------+------+------------+---------+------------+
|  10000035|  1552305125|         18| 567406|624183|     40|    49|19.843|      396.86| 1369.167|  2019-03-11|
|  10000098|  1551536152|         39| 816739|534452|     36|    45|12.157|      243.14|   790.14|  2019-03-02|
|  10000119|  1551093547|         25| 558284|157969|     30|    28|21.044|      420.88| 1010.112|  2019-02-25|
+----------+------------+-----------+-------+------+-------+------+------+------------+---------+------------+
only showing top 3 rows



In [31]:
#统计每个广告id每天的点击量
df_totalExposureLog.groupby(['ad_id','ad_rqst_date']).agg({'ad_size':'mean','ad_size':'count'}).show()

+------+------------+--------------+
| ad_id|ad_rqst_date|count(ad_size)|
+------+------------+--------------+
|568780|  2019-03-16|            46|
|318644|  2019-02-28|          1116|
|641438|  2019-02-20|          1445|
|145091|  2019-03-13|          1608|
| 31309|  2019-03-16|            29|
|218113|  2019-02-20|           929|
| 39673|  2019-03-12|            52|
|251983|  2019-03-04|          4612|
|200637|  2019-03-04|           130|
|695180|  2019-03-08|            33|
|431272|  2019-03-05|           444|
|579316|  2019-03-01|          6370|
|  8073|  2019-03-17|             1|
| 47454|  2019-02-21|           379|
|722803|  2019-03-05|            84|
|304105|  2019-02-26|           261|
|713698|  2019-02-26|            14|
|640759|  2019-03-14|          4079|
| 59825|  2019-02-27|           361|
|725160|  2019-03-01|          1466|
+------+------------+--------------+
only showing top 20 rows



In [32]:
group_tmp = df_totalExposureLog.groupby(['ad_id','ad_rqst_date']).agg({'ad_size':'mean','ad_size':'count'})

In [33]:
##统计周的曝光量
from pyspark.sql.functions import dayofweek
group_tmp.withColumn('exp_week',dayofweek(group_tmp.ad_rqst_date)).show()

+------+------------+--------------+--------+
| ad_id|ad_rqst_date|count(ad_size)|exp_week|
+------+------------+--------------+--------+
|568780|  2019-03-16|            46|       7|
|318644|  2019-02-28|          1116|       5|
|641438|  2019-02-20|          1445|       4|
|145091|  2019-03-13|          1608|       4|
| 31309|  2019-03-16|            29|       7|
|218113|  2019-02-20|           929|       4|
| 39673|  2019-03-12|            52|       3|
|251983|  2019-03-04|          4612|       2|
|200637|  2019-03-04|           130|       2|
|695180|  2019-03-08|            33|       6|
|431272|  2019-03-05|           444|       3|
|579316|  2019-03-01|          6370|       6|
|  8073|  2019-03-17|             1|       1|
| 47454|  2019-02-21|           379|       5|
|722803|  2019-03-05|            84|       3|
|304105|  2019-02-26|           261|       3|
|713698|  2019-02-26|            14|       3|
|640759|  2019-03-14|          4079|       5|
| 59825|  2019-02-27|           36

In [34]:
group_tmp = group_tmp.withColumn('exp_week',dayofweek(group_tmp.ad_rqst_date))
group_tmp.groupby('exp_week').count().sort('count',ascending=False).show()

+--------+------+
|exp_week| count|
+--------+------+
|       3|310520|
|       2|300426|
|       1|275312|
|       6|265771|
|       5|259297|
|       4|252358|
|       7|249832|
+--------+------+



In [35]:
group_tmp_pd = group_tmp.toPandas()
group_tmp_pd.to_csv(r'D:\AD_ALG\group_tmp_pd.csv',encoding='utf-8-sig',index=False,header=True)

### 第五步、表链接与构造训练集

In [36]:
df_ad_static= spark.read.csv(r'D:\\AD_ALG\\algo.qq.com_641013010_testa\\testA\\csvFile\\ad_static_feature.csv',header=True)
df_ad_operation= spark.read.csv(r'D:\AD_ALG\ad_operation_clear.csv',header=True)
df_ad_exposure = spark.read.csv(r'D:\AD_ALG\group_tmp_pd.csv',header=True)

In [37]:
df_ad_static.show(5)
df_ad_operation.show(5)
df_ad_exposure.show(5)

+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
| ad_id|create_time|ad_account_id|goods_id|goods_types|industry_id|ad_size|        create_date|
+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
|144732| 1550991780|        13937|      -1|          1|        186|     14|2019-02-24 15:03:00|
| 70793| 1552296016|         6932|      -1|          1|         21|     14|2019-03-11 17:20:16|
|467150| 1550823156|          171|   21922|         18|        232|     30|2019-02-22 16:12:36|
|383839| 1551188776|        10704|    4242|          8|        147|      3|2019-02-26 21:46:16|
|245464| 1550459042|        26317|   32825|          5|         47|     44|2019-02-18 11:04:02|
+------+-----------+-------------+--------+-----------+-----------+-------+-------------------+
only showing top 5 rows

+-----+------+--------+-------+-------+-------------------+--------------+--------------------+-----------+
|ad

In [38]:
df_ad_exposure = df_ad_exposure.withColumnRenamed('count(ad_size)','count')

In [39]:
#时间的转化
from pyspark.sql.functions import to_date
df_ad_static = df_ad_static.withColumn('create_date',to_date(df_ad_static.create_date))
df_ad_operation = df_ad_operation.withColumn('crt_chg_date',to_date(df_ad_operation.crt_chg_date))
df_ad_exposure = df_ad_exposure.withColumn('ad_rqst_date',to_date(df_ad_exposure.ad_rqst_date))

In [40]:
#合并df_ad_exposure和df_ad_static
df_ad_exposure = df_ad_exposure.withColumn('ad_id_key',df_ad_exposure.ad_id)
df_exp_sta = df_ad_exposure.join(df_ad_static, df_ad_exposure.ad_id == df_ad_static.ad_id, "inner")

In [41]:
df_exp_sta.show(10)

+------+------------+-----+--------+---------+------+-----------+-------------+--------+-----------+-----------+-------+-----------+
| ad_id|ad_rqst_date|count|exp_week|ad_id_key| ad_id|create_time|ad_account_id|goods_id|goods_types|industry_id|ad_size|create_date|
+------+------------+-----+--------+---------+------+-----------+-------------+--------+-----------+-----------+-------+-----------+
|100140|  2019-03-11|    3|       2|   100140|100140| 1551866001|        19104|   19626|         18|        117|     36| 2019-03-06|
|100263|  2019-03-11|   46|       2|   100263|100263| 1552215751|        27180|      -1|          1|        225|     36| 2019-03-10|
|100263|  2019-03-13|    9|       4|   100263|100263| 1552215751|        27180|      -1|          1|        225|     36| 2019-03-10|
|100263|  2019-03-12|   22|       3|   100263|100263| 1552215751|        27180|      -1|          1|        225|     36| 2019-03-10|
|100263|  2019-03-14|    1|       5|   100263|100263| 1552215751|    

In [42]:
#合并df_exp_sta和df_ad_operation.ad_id
df_ad_operation_id = df_ad_operation.withColumn('ad_id_key',df_ad_operation.ad_id).select('ad_id_key','put_time')
df_ad_operation_id = df_ad_operation_id.dropDuplicates(subset=['ad_id_key'])
df_exp_sta_opt = df_exp_sta.join(df_ad_operation_id, df_ad_operation_id.ad_id_key == df_exp_sta.ad_id_key, "inner")
df_exp_sta_opt.show(5)

+------+------------+-----+--------+---------+------+-----------+-------------+--------+-----------+-----------+-------+-----------+---------+--------------------+
| ad_id|ad_rqst_date|count|exp_week|ad_id_key| ad_id|create_time|ad_account_id|goods_id|goods_types|industry_id|ad_size|create_date|ad_id_key|            put_time|
+------+------------+-----+--------+---------+------+-----------+-------------+--------+-----------+-----------+-------+-----------+---------+--------------------+
|100320|  2019-03-03|   34|       1|   100320|100320| 1544437860|         1105|   10091|         13|         17|     30| 2018-12-10|   100320|281474976694272,2...|
|100320|  2019-03-14|    2|       5|   100320|100320| 1544437860|         1105|   10091|         13|         17|     30| 2018-12-10|   100320|281474976694272,2...|
|100320|  2019-03-05|   21|       3|   100320|100320| 1544437860|         1105|   10091|         13|         17|     30| 2018-12-10|   100320|281474976694272,2...|
|100320|  2019-0

In [43]:
df_exp_sta_opt = df_exp_sta_opt.drop('ad_id','ad_id_key')

In [44]:
df_exp_sta_opt.show()

+------------+-----+--------+-----------+-------------+--------+-----------+-----------+-------+-----------+--------------------+
|ad_rqst_date|count|exp_week|create_time|ad_account_id|goods_id|goods_types|industry_id|ad_size|create_date|            put_time|
+------------+-----+--------+-----------+-------------+--------+-----------+-----------+-------+-----------+--------------------+
|  2019-03-03|   34|       1| 1544437860|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-03-14|    2|       5| 1544437860|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-03-05|   21|       3| 1544437860|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-03-04|   12|       2| 1544437860|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-02-21|    9|       5| 1544437860|         1105|   10091|         13|         17|  

In [45]:
df_exp_sta_opt.count()

180104

In [46]:
df_exp_sta_opt_pd = df_exp_sta_opt.toPandas()
df_exp_sta_opt_pd.to_csv(r'D:\AD_ALG\df_exp_sta_opt_pd.csv',encoding='utf-8-sig',index=False,header=True)

### 第六步，处理测试集

In [47]:
df_test= spark.read.csv(r'D:\AD_ALG\testB.csv',header=True)
from pyspark.sql.functions import to_date
df_test = df_test.withColumn('pred_date',to_date(df_test.pred_date))

In [48]:
df_test.show()

+---------+------+-------+-----------+-----------+--------+-------------+--------------------+--------------------+------+----------+
|sample_id| ad_id|ad_size|industry_id|goods_types|goods_id|ad_account_id|         target_user|            put_time|ad_bid| pred_date|
+---------+------+-------+-----------+-----------+--------+-------------+--------------------+--------------------+------+----------+
|        1|152978|     30|        118|          3|      -1|        23511|age:182,202,739,3...|281,474,976,710,6...|    37|2019-03-21|
|        2|507977|     30|        122|         13|     629|         4211|                 all|70,368,744,161,28...|    54|2019-03-21|
|        3|639943|     34|         84|         13|    2525|        15582|age:433,202,739,3...|281,474,976,694,2...|   161|2019-03-21|
|        4|605859|     40|         98|         13|    3808|        10907|                 all|1,099,243,192,320...|    28|2019-03-22|
|        5|742582|     40|        122|         13|    7017|   

In [49]:
from pyspark.sql.functions import dayofweek
df_test=df_test.withColumn('exp_week',dayofweek(df_test.pred_date))

In [50]:
df_test.show(5)

+---------+------+-------+-----------+-----------+--------+-------------+--------------------+--------------------+------+----------+--------+
|sample_id| ad_id|ad_size|industry_id|goods_types|goods_id|ad_account_id|         target_user|            put_time|ad_bid| pred_date|exp_week|
+---------+------+-------+-----------+-----------+--------+-------------+--------------------+--------------------+------+----------+--------+
|        1|152978|     30|        118|          3|      -1|        23511|age:182,202,739,3...|281,474,976,710,6...|    37|2019-03-21|       5|
|        2|507977|     30|        122|         13|     629|         4211|                 all|70,368,744,161,28...|    54|2019-03-21|       5|
|        3|639943|     34|         84|         13|    2525|        15582|age:433,202,739,3...|281,474,976,694,2...|   161|2019-03-21|       5|
|        4|605859|     40|         98|         13|    3808|        10907|                 all|1,099,243,192,320...|    28|2019-03-22|       6|

In [51]:
df_test_pd = df_test.toPandas()
df_test_pd.to_csv(r'D:\AD_ALG\df_test_pd.csv',encoding='utf-8-sig',index=False,header=True)

### 第七步，初步建模，baseline

In [52]:
df_exp_sta_opt = spark.read.csv(r'D:\AD_ALG\df_exp_sta_opt_pd.csv',header=True)
df_train = df_exp_sta_opt.drop('create_time')
df_train.show()

+------------+-----+--------+-------------+--------+-----------+-----------+-------+-----------+--------------------+
|ad_rqst_date|count|exp_week|ad_account_id|goods_id|goods_types|industry_id|ad_size|create_date|            put_time|
+------------+-----+--------+-------------+--------+-----------+-----------+-------+-----------+--------------------+
|  2019-03-03|   34|       1|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-03-14|    2|       5|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-03-05|   21|       3|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-03-04|   12|       2|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-02-21|    9|       5|         1105|   10091|         13|         17|     30| 2018-12-10|281474976694272,2...|
|  2019-02-23|    1|       7|         1105|   10091|    

In [53]:
df_train.select('ad_rqst_date').dropDuplicates().show()

+------------+
|ad_rqst_date|
+------------+
|  2019-03-03|
|  2019-03-11|
|  2019-02-24|
|  2019-03-07|
|  2019-03-19|
|  2019-03-01|
|  2019-02-19|
|  2019-02-28|
|  2019-02-26|
|  2019-02-21|
|  2019-02-23|
|  2019-02-17|
|  2019-03-16|
|  2019-03-05|
|  2019-02-25|
|  2019-02-22|
|  2019-03-14|
|  2019-02-27|
|  2019-02-20|
|  2019-03-15|
+------------+
only showing top 20 rows



In [54]:
#以3月19日之前曝光的广告作为新的训练集，3月19日的广告作为测试集
df_train_tmp = df_train.where(df_train.ad_rqst_date == '2019-03-19')
df_test_tmp = df_train.where(df_train.ad_rqst_date != '2019-03-19')

In [55]:
#选择特征
train_cols = ['count','exp_week','ad_account_id','goods_id','goods_types','industry_id','ad_size']
features = ['exp_week','ad_account_id','goods_id','goods_types','industry_id','ad_size']
df_trainA = df_train_tmp.select(train_cols)
df_testA = df_test_tmp.select(train_cols)
#真正的验证集
df_testB = spark.read.csv(r'D:\AD_ALG\df_test_pd.csv',header=True).select(features)

In [56]:
#转换数据类型
import pyspark.sql.types as typ
for col in train_cols:
    df_trainA = df_trainA.withColumn(col,df_train[col].cast(typ.IntegerType()))
    df_testA = df_testA.withColumn(col,df_testA[col].cast(typ.IntegerType()))
    if col != 'count':
        df_testB = df_testB.withColumn(col,df_testB[col].cast(typ.IntegerType()))

In [57]:
##建立梯度提升模型
import pyspark.ml.regression as reg
from pyspark.ml import Pipeline
import pyspark.ml.feature as ft
featuresCreator = ft.VectorAssembler(
    inputCols = [col for col in features],
    outputCol = 'features'
)
selector = ft.ChiSqSelector(
    numTopFeatures = 6,
    outputCol = 'selectedFeatures',
    labelCol= 'count'
)
gbt = reg.GBTRegressor(
maxIter = 15,
maxDepth=3,
seed=617,
labelCol = 'count'
)
pipeline = Pipeline(stages=[
    featuresCreator,
    selector,
    gbt 
])
gbt_model = pipeline.fit(df_trainA) 

In [58]:
##评估模型
import pyspark.ml.evaluation as ev
evaluator = ev.RegressionEvaluator(
    predictionCol = 'prediction',
    labelCol = 'count',
)

print('梯度提升预测r2为：',evaluator.evaluate(
    gbt_model.transform(df_testA),
    {evaluator.metricName:'r2'}
))

梯度提升预测r2为： 0.10008700437076656


In [83]:
#做预测
gbt_pred = gbt_model.transform(df_testB).toPandas()

In [64]:
#建立随机森林模型
rfr = reg.RandomForestRegressor(numTrees=10, labelCol = 'count')
pipeline2 = Pipeline(stages=[
    featuresCreator,
    selector,
    rfr 
])
rfr_model = pipeline2.fit(df_trainA) 
##评估模型
import pyspark.ml.evaluation as ev
evaluator = ev.RegressionEvaluator(
    predictionCol = 'prediction',
    labelCol = 'count',
)

print('随机森林预测r2为：',evaluator.evaluate(
    rfr_model.transform(df_testA),
    {evaluator.metricName:'r2'}
))

随机森林预测r2为： 0.0772248218333691


In [84]:
#做预测
rfr_pred = rfr_model.transform(df_testB).toPandas()

In [85]:
##模型融合及调整单调性
df_pred = spark.read.csv(r'D:\AD_ALG\df_test_pd.csv',header=True).select('ad_id','ad_bid').toPandas()

In [87]:
df_pred['gbt_pred'] = gbt_pred['prediction']
df_pred['rfr_pred'] = rfr_pred['prediction']
df_pred['avg_pred'] = (rfr_pred['prediction']+ gbt_pred['prediction'])/2
df_pred.head()

,ad_id,ad_bid,gbt_pred,rfr_pred,avg_pred
0,152978,37,59.272329,88.748545,74.010437
1,507977,54,13.170180,10.313342,11.741761
2,639943,161,7.191413,8.432640,7.812026
3,605859,28,19.472607,20.880307,20.176457
4,742582,12,19.472607,21.632217,20.552412


In [88]:
df_pred.avg_pred.describe()

count    38596.000000
mean        58.330360
std         69.786789
min        -76.226183
25%         11.576575
50%         30.355260
75%         79.048574
max        708.886150
Name: avg_pred, dtype: float64

In [90]:
df_pred['finall_pred'] = df_pred['avg_pred']+df_pred['ad_bid'].astype(int)/10000

In [91]:
df_pred.head()

,ad_id,ad_bid,gbt_pred,rfr_pred,avg_pred,finall_pred
0,152978,37,59.272329,88.748545,74.010437,74.014137
1,507977,54,13.170180,10.313342,11.741761,11.747161
2,639943,161,7.191413,8.432640,7.812026,7.828126
3,605859,28,19.472607,20.880307,20.176457,20.179257
4,742582,12,19.472607,21.632217,20.552412,20.553612


### 第八步，增加特征建模

In [ ]:
df_exp_sta_opt = spark.read.csv(r'D:\AD_ALG\df_exp_sta_opt_pd.csv',header=True)
df_train = df_exp_sta_opt.drop('create_time')
df_test = spark.read.csv(r'D:\AD_ALG\df_test_pd.csv',header=True)